# Tensorflow2.0 based code

In [1]:
import tensorflow as tf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
path = '/home/blu/workspace/graduate_project/data/'

## 一共有45148条路，每条路5个特征

In [19]:
road_network = pd.read_table(path+'road_network_sub-dataset.v2')
road_network.head()

,link_id,width,direction,snodeid,enodeid,length,speedclass,lanenum
0,1562548955,30,3,1520445066,1549742690,0.038,6,1
1,1570012174,55,2,1520482305,1554157827,0.013,5,2
2,1542179399,30,2,1520491527,1531173821,0.146,6,1
3,1562548950,55,1,1520433538,1549742156,0.029,7,1
4,1526068509,30,2,1520486391,1520499209,0.156,6,1


In [7]:
road_network.shape

(45148, 8)

## Feature engineering

- 连续变量

item | min | max | mean | std | unit
:-:|:-:|:-:|:-:|:-:|:-:
length | 0.002 | 0.492 | 0.0588 | 0.063 | 米


- 离散变量

item | value | meanings |
:-:|:-:|:-:
width | {15,30,55} | 15：小于等于3m 30：3m至5m之间 55：5.5m至13m之间
direction | {1,2,3} | 1：双向 2：单向（s->e）3：单向（e->s）
speedclass | {2,3,4,5,6,7,8} | 实际上是没有1的，速度class越小越快
lanenum | {1,2,3} | 1：单车道 2：双或三车道 3：多车道

In [13]:
def width_label_change(x):
    if(x['width'] == 15):
        return 0
    elif(x['width'] == 30):
        return 1
    else:
        return 2
rr = road_network.apply(lambda x: width_label_change(x), axis=1)
road_network["width_label"] = rr

In [22]:
def direction_label_change(x):
    x['direction'] = x['direction'] - 1
    return x['direction']
rr = road_network.apply(lambda x: direction_label_change(x), axis=1)
road_network["direction"] = rr

In [25]:
def speedclass_label_change(x):
    x['speedclass'] = x['speedclass'] - 2
    return x['speedclass']
rr = road_network.apply(lambda x: speedclass_label_change(x), axis=1)
road_network["speedclass"] = rr

def lanenum_label_change(x):
    x['lanenum'] = x['lanenum'] - 1
    return x['lanenum']
ll = road_network.apply(lambda x: lanenum_label_change(x), axis=1)
road_network["lanenum"] = ll

In [51]:
road_network.head()

,link_id,width,direction,snodeid,enodeid,length,speedclass,lanenum,length_label
0,1562548955,30,2.0,1520445066,1549742690,0.038,4.0,0.0,1
1,1570012174,55,1.0,1520482305,1554157827,0.013,3.0,1.0,0
2,1542179399,30,1.0,1520491527,1531173821,0.146,4.0,0.0,3
3,1562548950,55,0.0,1520433538,1549742156,0.029,5.0,0.0,1
4,1526068509,30,1.0,1520486391,1520499209,0.156,4.0,0.0,3


In [15]:
width_oh = tf.keras.utils.to_categorical(road_network['width_label'], num_classes = 3)
print(width_oh[0:5])

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [24]:
direction_oh = tf.keras.utils.to_categorical(road_network['direction'], num_classes = 3)
print(direction_oh[0:5])

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [27]:
speedclass_oh = tf.keras.utils.to_categorical(road_network['speedclass'], num_classes = 7)
lanenum_oh =  tf.keras.utils.to_categorical(road_network['lanenum'], num_classes = 3)

In [29]:
road_network.describe()['length']

count    45148.000000
mean         0.058849
std          0.063129
min          0.002000
25%          0.016000
50%          0.038000
75%          0.077000
max          0.492000
Name: length, dtype: float64

In [50]:
def length_label_change(x):
    length = int(x['length']*100)
    if(length <= 1):
        return 0
    elif(length >= 2 and length <= 3):
        return 1
    elif(length >= 4 and length <= 7):
        return 2
    else:
        return 3
rr = road_network.apply(lambda x: length_label_change(x), axis=1)
road_network["length_label"] = rr

In [52]:
length_oh =  tf.keras.utils.to_categorical(road_network['length_label'], num_classes = 4)

In [59]:
print("width:", width_oh.shape)
print("direction:", direction_oh.shape)
print("length:", length_oh.shape)
print("speedclass:", speedclass_oh.shape)
print("lanenum: ", lanenum_oh.shape)


width: (45148, 3)
direction: (45148, 3)
length: (45148, 4)
speedclass: (45148, 7)
lanenum:  (45148, 3)


In [64]:
feature_oh = tf.concat((width_oh, direction_oh, length_oh, speedclass_oh, lanenum_oh),axis=1)
feature_oh.shape

TensorShape([45148, 20])

In [67]:
feature_oh.numpy()

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]], dtype=float32)

In [68]:
feature_matrix = pd.DataFrame(feature_oh.numpy())
feature_matrix.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [69]:
feature_matrix["link_id"] = road_network["link_id"]
feature_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,link_id
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1562548955
1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1570012174
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1542179399
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1562548950
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1526068509


In [71]:
cols = list(feature_matrix)
cols.insert(0,cols.pop(cols.index('link_id')))
feature_matrix = feature_matrix.loc[:,cols]
feature_matrix.head()

,link_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,1562548955,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1570012174,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1542179399,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1562548950,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1526068509,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [75]:
save_path = '/home/blu/workspace/graduate_project/STGCN/beijing_traffic/01_road_feature_matrix_full.npy'
np.save(save_path, feature_matrix.values)